In [1]:
import polars as pl

import rustystats as rs

data = pl.read_parquet("https://raw.githubusercontent.com/PricingFrontier/pricing-data-example/917c853e256df8d5814721ab56f72889a908bb08/data/processed/frequency_set.parquet")

train_data = data.filter(pl.col("Group") <= "3")
test_data = data.filter(pl.col("Group") == "4")
holdout_data = data.filter(pl.col("Group") == "5")

In [2]:
terms = {
        'DrivAge': {'type': 'linear'}, 
        'VehPower': {'type': 'linear'}
    }

interactions=[{
        "VehPower": {"type": "linear"},
        "Region": {"type": "categorical"},
        "target_encoding": True,
        "include_main": False
    }]

result = rs.glm_dict(
    response='ClaimCount',
    terms=terms,
    interactions=interactions,
    data=data,
    family='poisson',
    offset ='Exposure'
).fit(store_design_matrix=False)

result.diagnostics(
    train_data=train_data,
    test_data = test_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

ModelDiagnostics(model_summary={'formula': 'ClaimCount ~ DrivAge + VehPower', 'family': 'poisson', 'link': 'log', 'n_obs': 406439, 'n_params': 4, 'df_resid': 406435, 'converged': True, 'iterations': 6}, train_test=TrainTestComparison(train=DatasetDiagnostics(dataset='train', n_obs=406439, total_exposure=214930.9, total_actual=21408.0, total_predicted=21259.76, loss=0.324012, deviance=131691.0, log_likelihood=-86619.32, aic=173246.65, gini=0.0948, auc=0.5474, ae_ratio=1.007, ae_by_decile=[DecileMetrics(decile=1, n=40643, exposure=1590.4, actual=0.718059, predicted=0.097673, ae_ratio=7.3516), DecileMetrics(decile=2, n=40643, exposure=3518.67, actual=0.293008, predicted=0.102444, ae_ratio=2.8602), DecileMetrics(decile=3, n=40643, exposure=7190.7, actual=0.210411, predicted=0.098924, ae_ratio=2.127), DecileMetrics(decile=4, n=40643, exposure=11705.47, actual=0.144377, predicted=0.100187, ae_ratio=1.4411), DecileMetrics(decile=5, n=40643, exposure=17734.46, actual=0.116722, predicted=0.0984

In [ ]:
#writes analysis/diagnostics


ModelDiagnostics(model_summary={'formula': 'ClaimCount ~ ns(BonusMalus, df=10) + ns(VehAge, df=10) + ns(DrivAge, df=10) + ns(VehPower, df=10) + TE(Area) + TE(Region) + TE(VehGas) + TE(VehBrand)', 'family': 'poisson', 'link': 'log', 'n_obs': 406439, 'n_params': 42, 'df_resid': 406397, 'converged': True, 'iterations': 7}, train_test=TrainTestComparison(train=DatasetDiagnostics(dataset='train', n_obs=406439, total_exposure=214930.9, total_actual=21408.0, total_predicted=21299.38, loss=0.309535, deviance=125807.27, log_likelihood=-83677.46, aic=167438.92, gini=0.2839, auc=0.642, ae_ratio=1.0051, ae_by_decile=[DecileMetrics(decile=1, n=40643, exposure=1925.48, actual=0.390032, predicted=0.075343, ae_ratio=5.1768), DecileMetrics(decile=2, n=40643, exposure=4241.32, actual=0.224458, predicted=0.088789, ae_ratio=2.528), DecileMetrics(decile=3, n=40643, exposure=8285.64, actual=0.163536, predicted=0.084522, ae_ratio=1.9348), DecileMetrics(decile=4, n=40643, exposure=14058.11, actual=0.119717, p

In [4]:
print(result.summary())

                                 GLM Results                                  

Family:              Poisson         No. Observations:        678012
Link Function:       (default)       Df Residuals:            677970
Method:              IRLS + Smooth   Df Model:                    41
Scale:               1.0000          Alpha (λ):               0.0000
L1 Ratio:            0.00            Iterations:                   7
Non-zero coefs:      41             

Log-Likelihood:         -139169.0199 Deviance:                209337.8937
AIC:                     278422.0398 Null Deviance:           220244.0296
BIC:                     278901.9705 Pearson chi2:             1521969.76
Converged:           True           

------------------------------------------------------------------------------
Variable                  Coef    Std.Err        z    P>|z|                 95% CI     
------------------------------------------------------------------------------
Intercept              -3.1811 

In [5]:
result.predict(holdout_data)

array([0.05153975, 0.04878665, 0.04929043, ..., 0.00560929, 0.0173981 ,
       0.07110551], shape=(135957,))

In [6]:
# Model spec before interaction
terms = {
    "BonusMalus": {"type": "linear"},
    "VehAge": {"type": "ns", "df": 5},
    "DrivAge": {"type": "linear"},
    "Region": {"type": "target_encoding"},
    "VehBrand": {"type": "target_encoding"},
    "Area": {"type": "target_encoding"},
}
# Interaction to add (DrivAge x BonusMalus)
interactions = [
    {
        "DrivAge": {"type": "linear"},
        "Region": {"type": "target_encoding"},
        "include_main": False,
    }
]
# Fit with interaction
result = rs.glm_dict(
    response="ClaimCount",
    terms=terms,
    interactions=interactions,
    data=train_data,
    family="poisson",
    offset="Exposure",
).fit()


In [7]:
print(result.summary())

                                 GLM Results                                  

Family:              Poisson         No. Observations:        406439
Link Function:       (default)       Df Residuals:            406428
Method:              IRLS            Df Model:                    10
Scale:               1.0000          Iterations:                   6

Log-Likelihood:          -84120.8663 Deviance:                126694.0829
AIC:                     168263.7326 Null Deviance:           132328.3090
BIC:                     168383.7997 Pearson chi2:              934134.22
Converged:           True           

------------------------------------------------------------------------------
Variable                 Coef    Std.Err        z    P>|z|                 95% CI     
------------------------------------------------------------------------------
Intercept             -4.4165     0.1591  -27.761  <0.0001   [ -4.7283,  -4.1047]  ***
BonusMalus             0.0219     0.0004   54.606  

In [8]:
#creates analysis/exploration.json
exploration = rs.explore_data(
    data=train_data,
    response="ClaimCount",
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ],
    exposure="Exposure",
)

In [9]:
#writes analysis/diagnostics
result.diagnostics(
    train_data=train_data,
    test_data = test_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

ModelDiagnostics(model_summary={'formula': 'ClaimCount ~ BonusMalus + ns(VehAge, df=5) + DrivAge + TE(Region) + TE(VehBrand) + TE(Area)', 'family': 'poisson', 'link': 'log', 'n_obs': 406439, 'n_params': 11, 'df_resid': 406428, 'converged': True, 'iterations': 6}, train_test=TrainTestComparison(train=DatasetDiagnostics(dataset='train', n_obs=406439, total_exposure=214930.9, total_actual=21408.0, total_predicted=21423.97, loss=0.31169, deviance=126682.82, log_likelihood=-84115.24, aic=168252.47, gini=0.2595, auc=0.6297, ae_ratio=0.9993, ae_by_decile=[DecileMetrics(decile=1, n=40643, exposure=1904.21, actual=0.412245, predicted=0.077997, ae_ratio=5.2854), DecileMetrics(decile=2, n=40643, exposure=4177.68, actual=0.235777, predicted=0.091478, ae_ratio=2.5774), DecileMetrics(decile=3, n=40643, exposure=8113.67, actual=0.165523, predicted=0.087567, ae_ratio=1.8902), DecileMetrics(decile=4, n=40643, exposure=13741.07, actual=0.127137, predicted=0.083094, ae_ratio=1.53), DecileMetrics(decile=5

In [10]:
predictions = result.predict(holdout_data)

In [11]:
model_bytes = result.to_bytes()
with open("model.bin", "wb") as f:
    f.write(model_bytes)

In [12]:
with open("model.bin", "rb") as f:
    loaded = rs.GLMModel.from_bytes(f.read())

In [13]:
predictions2 = loaded.predict(holdout_data)

In [14]:
import numpy as np
print(np.array_equal(predictions2, predictions))

True


In [15]:
#writes analysis/diagnostics
loaded.diagnostics(
    train_data=train_data,
    test_data = train_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

ModelDiagnostics(model_summary={'formula': 'ClaimCount ~ BonusMalus + ns(VehAge, df=5) + DrivAge + TE(Region) + TE(VehBrand) + TE(Area)', 'family': 'poisson', 'link': 'log', 'n_obs': 406439, 'n_params': 11, 'df_resid': 406428, 'converged': True, 'iterations': 6}, train_test=TrainTestComparison(train=DatasetDiagnostics(dataset='train', n_obs=406439, total_exposure=214930.9, total_actual=21408.0, total_predicted=21423.97, loss=0.31169, deviance=126682.82, log_likelihood=-84115.24, aic=168252.47, gini=0.2595, auc=0.6297, ae_ratio=0.9993, ae_by_decile=[DecileMetrics(decile=1, n=40643, exposure=1904.21, actual=0.412245, predicted=0.077997, ae_ratio=5.2854), DecileMetrics(decile=2, n=40643, exposure=4177.68, actual=0.235777, predicted=0.091478, ae_ratio=2.5774), DecileMetrics(decile=3, n=40643, exposure=8113.67, actual=0.165523, predicted=0.087567, ae_ratio=1.8902), DecileMetrics(decile=4, n=40643, exposure=13741.07, actual=0.127137, predicted=0.083094, ae_ratio=1.53), DecileMetrics(decile=5

In [16]:
result2 = rs.glm_dict(
    response="ClaimCount",
    terms={
        "Region": {"type": "target_encoding"},
        "BonusMalus": {"type": "linear"},
        "BonusMalus2": {"type": "expression", "expr": "BonusMalus ** 2"}
    },
    data=train_data,
    family="poisson",
    offset="Exposure",
).fit(
    regularization="elastic_net"
)

In [17]:
#writes analysis/diagnostics
result2.diagnostics(
    train_data=train_data,
    test_data = train_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

ModelDiagnostics(model_summary={'formula': 'ClaimCount ~ TE(Region) + BonusMalus + I(BonusMalus ** 2)', 'family': 'poisson', 'link': 'log', 'n_obs': 406439, 'n_params': 4, 'df_resid': 406435, 'converged': True, 'iterations': 6}, train_test=TrainTestComparison(train=DatasetDiagnostics(dataset='train', n_obs=406439, total_exposure=214930.9, total_actual=21408.0, total_predicted=21376.81, loss=0.318672, deviance=129520.81, log_likelihood=-85534.23, aic=171076.46, gini=0.1832, auc=0.5916, ae_ratio=1.0015, ae_by_decile=[DecileMetrics(decile=1, n=40643, exposure=1695.84, actual=0.640391, predicted=0.088785, ae_ratio=7.2128), DecileMetrics(decile=2, n=40643, exposure=3744.33, actual=0.288703, predicted=0.099021, ae_ratio=2.9156), DecileMetrics(decile=3, n=40643, exposure=7519.51, actual=0.182725, predicted=0.094867, ae_ratio=1.9261), DecileMetrics(decile=4, n=40643, exposure=12492.93, actual=0.139199, predicted=0.095028, ae_ratio=1.4648), DecileMetrics(decile=5, n=40643, exposure=18491.42, ac

In [18]:
result3 = rs.glm_dict(
    response="ClaimCount",
    terms={
        "BonusMalus": {"type": "ns"},
        "VehAge": {"type": "ns"},
        "DrivAge": {"type": "linear"},
        "VehBrand": {"type": "target_encoding"}
    },
    interactions= [
        {
            "VehAge": {"type": "linear"},
            "BonusMalus": {"type": "linear"},
            "include_main": False
        },
        {
            "DrivAge": {"type": "linear"},
            "BonusMalus": {"type": "linear"},
            "include_main": False
        }
    ],
    data=train_data,
    family="poisson",
    offset="Exposure",
).fit(
    regularization="elastic_net",
    include_unregularized=False
)

In [19]:
import rustystats as rs

# Model spec that caused the issue when elastic_net was applied
terms = {
    "BonusMalus": {'type' : "linear"},
    "VehAge": {'type' : "ns"}, 
    "DrivAge": {'type' : "ns"},
    "Area": {'type' : "target_encoding"},
}

interactions= [
        {
            "DrivAge": {"type": "linear"},
            "BonusMalus": {"type": "linear"},
            "include_main": False
        }
    ]

# Build the model
glm = rs.glm_dict(
    response="ClaimCount",
    terms=terms,
    interactions=interactions,
    data=train_data,  # freMTPL2freq dataset
    family="poisson",
    offset="Exposure",  # log-linked
)

# This is where the issue occurs - regularized fit + diagnostics
result = glm.fit()

# Check if diagnostics returns None for deviance
diag = result.diagnostics(train_data=train_data, test_data=holdout_data)
print(diag.to_dict())  # Check train_test.test.deviance

{'model_summary': {'formula': 'ClaimCount ~ BonusMalus + ns(VehAge, df=10) + ns(DrivAge, df=10) + TE(Area)', 'family': 'poisson', 'link': 'log', 'n_obs': 406439, 'n_params': 22, 'df_resid': 406417, 'converged': True, 'iterations': 7}, 'train_test': {'train': {'dataset': 'train', 'n_obs': 406439, 'total_exposure': 214930.9, 'total_actual': 21408.0, 'total_predicted': 21365.75, 'loss': 0.310818, 'deviance': 126328.72, 'log_likelihood': -83938.18, 'aic': 167920.36, 'gini': 0.2737, 'auc': 0.6368, 'ae_ratio': 1.002, 'ae_by_decile': [{'decile': 1, 'n': 40643, 'exposure': 1903.47, 'actual': 0.39717, 'predicted': 0.077225, 'ae_ratio': 5.143}, {'decile': 2, 'n': 40643, 'exposure': 4158.05, 'actual': 0.240257, 'predicted': 0.090654, 'ae_ratio': 2.6503}, {'decile': 3, 'n': 40643, 'exposure': 8129.94, 'actual': 0.164331, 'predicted': 0.086765, 'ae_ratio': 1.894}, {'decile': 4, 'n': 40643, 'exposure': 13702.9, 'actual': 0.123769, 'predicted': 0.082521, 'ae_ratio': 1.4998}, {'decile': 5, 'n': 40643,